<a href="https://colab.research.google.com/github/federicovolponi/SLNN/blob/main/CNN/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - Cats vs. Dogs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Unzip data

In [ ]:
!unzip /content/drive/MyDrive/ColabNotebooks/Federico/Neuralnetworks/Lab2/train.zip -d train/
!unzip /content/drive/MyDrive/ColabNotebooks/Federico/Neuralnetworks/Lab2/test.zip -d test/

Code to initiliaze Tensorflow 2.0 in Colab

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
%load_ext tensorboard
import datetime
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

**[TODO] Create a data loader function that returns a tuple with a tf.float32 tensor for the image and a label. Images must be resized to 128x128.**
**N.B.: filenames are formatted as class.number.jpg**

In [ ]:
def loader(filename):
  label = tf.strings.to_number(tf.strings.split(tf.strings.split(filename, '.')[0], '/')[-1])
  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [128, 128])
  return image, label

**[TODO] Create a tf.Dataset, map the loader function and prepare a batch object for training**

In [ ]:
img_train = tf.data.Dataset.list_files("/content/train/*.jpg")
img_test = tf.data.Dataset.list_files("/content/test/*.jpg")
img_dataset = img_train.map(loader).batch(16)
img_testSet = img_test.map(loader).batch(16)

**Prepare Keras callback for Tensorboard**

In [ ]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
%tensorboard --logdir logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq='batch')

**[TODO] Import the MobileNetV2 model trained on ImageNet without the final layer**

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3), include_top=False, weights='imagenet')
base_model.trainable = False

**[TODO] Add a final classification layer for 2 classes and create the final Keras model**

In [ ]:
x = base_model.input
h = base_model(x)
y = tf.keras.layers.Flatten()(h)
y = tf.keras.layers.Dense(2 , activation='softmax')(y)
model = tf.keras.Model(inputs=x, outputs=y)

**[TODO] Compile the Keras model: specify the optimization algorithm, the loss function and the test metric**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**[TODO] Train the Keras model**

In [ ]:
model.fit(img_dataset, epochs=5, callbacks=tensorboard_callback)

**[TODO] Print model summary**

In [ ]:
model.summary()

**[TODO] Test the Keras model by computing the accuracy the whole test set**

In [ ]:
model.evaluate(img_testSet)

**[TODO] Load Test image 'test/0.47.jpg', visualize it and check the network prediction**

In [ ]:
animals = ['cat', 'dog']
img, lab = loader("/content/test/0.1047.jpg")
plt.imshow(img)
pred = model.predict(tf.expand_dims(img,0))
print("This is a", animals[np.argmax(pred)])